## Importing controller

This cell imports the learnt controller $nncontrol.pt$


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

n = 3
control_dim = 2

input_dim = n+1


class NNControl(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 150,bias=True)  # 2 input features, 5 hidden units
        self.fc2 = nn.Linear(150,300,bias=True)
#         self.fc3 = nn.Linear(300,200,bias=True)
        self.fc3 = nn.Linear(300,20,bias=True)
        self.relu = nn.ReLU()

        self.out = nn.Linear(20, control_dim)  # 5 hidden units, 1 output



    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)

        x = self.fc2(x)
        x = self.relu(x)

        x = self.fc3(x)
        x = self.relu(x)    
        
#         x = self.fc4(x)
#         x = self.relu(x) 

        x = self.out(x)
        
        

        return x



nncontrol = NNControl()

nncontrol.load_state_dict(torch.load('nncontrol.pt'))




<All keys matched successfully>

## Converts unicycle control input to differential drive input

In [2]:
def unicycle2diffdrive(v,w,R,L):
    Vr = (v + L*w/2)/R 
    Vl = (v - L*w/2)/R
    
    speeds = [Vr,Vl]
    return speeds


## PyBullet environment with one Husky robot

In [4]:
import pybullet as p
import time
import pybullet_data
physicsClient = p.connect(p.GUI)#or p.DIRECT for non-graphical version
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)
p.configureDebugVisualizer(lightPosition=[10,-100,250])
# print(p.getDebugVisualizerCamera(physicsClient))
p.resetDebugVisualizerCamera(cameraDistance=5,cameraYaw=150,cameraPitch=-30,cameraTargetPosition=[5,2,2])

print(pybullet_data.getDataPath())

p.setAdditionalSearchPath(pybullet_data.getDataPath()) #optionally
p.setGravity(0,0,-9.8)
planeId = p.loadURDF("plane.urdf")
startPos = [-4,-4,0.45]
startOrientation = p.getQuaternionFromEuler([0,0,2])
time.sleep(2.)

# boxId = p.loadURDF("r2d2.urdf",startPos, startOrientation)
cube = p.loadURDF("blk_cube.urdf",[0,0,0.5],startOrientation)
time.sleep(1.)
# boxId = p.loadURDF("husky/husky.urdf",startPos, startOrientation)

boxId = p.loadURDF("r2d2.urdf",startPos, startOrientation)

time.sleep(1.)

# boxId = p.loadURDF("biped/biped2d_pybullet.urdf",startPos, startOrientation)

# boxId = p.loadURDF("racecar/racecar_differential.urdf",startPos, startOrientation)

#set the center of mass frame (loadURDF sets base link frame) startPos/Ornp.resetBasePositionAndOrientation(boxId, startPos, startOrientation)
print(p.getNumJoints(boxId))
# for i in range(10):
#     print(p.getJointInfo(boxId, jointIndex=i))
#     print("")

# time.sleep(1)
# p.setRealTimeSimulation(0)
logger = p.startStateLogging(p.STATE_LOGGING_VIDEO_MP4,fileName = 'r2d2_video.mp4')

maxForce = 500
x = 10

# time.sleep(5)
left = 0
right = 0
for i in range(3000):
    cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)
    orientation = p.getEulerFromQuaternion(cubeOrn)
    pos = torch.tensor([cubePos[0],cubePos[1],orientation[2],i/50])
    
#     print(cubePos[2])
    
    unispeeds = nncontrol(pos) * 3.5
       

    
    #### HUSKY ####
#     right,left = unicycle2diffdrive(unispeeds[0],unispeeds[1],1.1,0.71)    
#     p.setJointMotorControl2(boxId,2,controlMode=p.VELOCITY_CONTROL,targetVelocity = left,force=maxForce)
#     p.setJointMotorControl2(boxId,3,controlMode=p.VELOCITY_CONTROL,targetVelocity = right,force=maxForce)
    
#     p.setJointMotorControl2(boxId,4,controlMode=p.VELOCITY_CONTROL,targetVelocity = left,force=maxForce)
#     p.setJointMotorControl2(boxId,5,controlMode=p.VELOCITY_CONTROL,targetVelocity = right,force=maxForce)    
        
    #### R2D2 ####
    right,left = unicycle2diffdrive(unispeeds[0],unispeeds[1],3.7,0.31)    
    p.setJointMotorControl2(boxId,2,controlMode=p.VELOCITY_CONTROL,targetVelocity = right,force=maxForce)
    p.setJointMotorControl2(boxId,3,controlMode=p.VELOCITY_CONTROL,targetVelocity = right,force=maxForce)
    
    p.setJointMotorControl2(boxId,6,controlMode=p.VELOCITY_CONTROL,targetVelocity = left,force=maxForce)
    p.setJointMotorControl2(boxId,7,controlMode=p.VELOCITY_CONTROL,targetVelocity = left,force=maxForce)
    
    p.stepSimulation()
    time.sleep(1./100.)


p.stopStateLogging(logger)    
p.disconnect()

C:\Users\dagadginmath\AppData\Local\anaconda3\envs\pytorch_env\Lib\site-packages\pybullet_data
15


## Pybullet environment with four Husky robots

In [5]:
# import pybullet as p
# import time
# import pybullet_data
# physicsClient = p.connect(p.GUI)#or p.DIRECT for non-graphical version
# p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)
# p.configureDebugVisualizer(lightPosition=[10,-100,250])
# # print(p.getDebugVisualizerCamera(physicsClient))
# p.resetDebugVisualizerCamera(cameraDistance=15,cameraYaw=60,cameraPitch=-30,cameraTargetPosition=[10,2,2])

# # print(pybullet_data.getDataPath())

# p.setAdditionalSearchPath(pybullet_data.getDataPath()) #optionally
# p.setGravity(0,0,-9.8)
# planeId = p.loadURDF("plane.urdf")
# # startPos = [6,6,0.005]
# startOrientation = p.getQuaternionFromEuler([0,0,2])
# time.sleep(2.)

# cube = p.loadURDF("blk_cube.urdf",[0,0,0.5],startOrientation)
# time.sleep(1.)

# num_agents = 4
# startPos = torch.normal(0,7,(num_agents,3)) * torch.tensor([1,1,0.000]) + torch.tensor([-2,2,0.005])
# boxId = [p.loadURDF("husky/husky.urdf",startPos[i,:], startOrientation) for i in range(num_agents)]

# time.sleep(1.)

# logger = p.startStateLogging(p.STATE_LOGGING_VIDEO_MP4,fileName = 'husky_video_multiple.mp4')

# maxForce = 50

# for i in range(3000):
#     for k in range(num_agents):
#         cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId[k])
#         orientation = p.getEulerFromQuaternion(cubeOrn)
#         pos = torch.tensor([cubePos[0],cubePos[1],orientation[2],i/300])
    
    
#         unispeeds = nncontrol(pos) * 1
       
#         right,left = unicycle2diffdrive(unispeeds[0],unispeeds[1],1.1,0.71)

#         p.setJointMotorControl2(boxId[k],2,controlMode=p.VELOCITY_CONTROL,targetVelocity = left,force=maxForce)
#         p.setJointMotorControl2(boxId[k],3,controlMode=p.VELOCITY_CONTROL,targetVelocity = right,force=maxForce)

#         p.setJointMotorControl2(boxId[k],4,controlMode=p.VELOCITY_CONTROL,targetVelocity = left,force=maxForce)
#         p.setJointMotorControl2(boxId[k],5,controlMode=p.VELOCITY_CONTROL,targetVelocity = right,force=maxForce)    
       
#     p.stepSimulation()
#     time.sleep(1./100.)


# p.stopStateLogging(logger)    
# p.disconnect()